In [1]:
%pylab widget
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

Populating the interactive namespace from numpy and matplotlib


In [2]:
import fastcat as fc
from glob import glob

## First I'll look at the detector MTF

### I'll load all of the kernels 

In [3]:
spectra = glob('../../fastcat/data/MV_spectra/*.txt')
phantoms = ['XCAT','XCAT2','Catphan_515','Catphan_MTF']
MV_detectors = glob('../../fastcat/data/Detectors/*')

dispatcher={'Catphan_515':fc.Catphan_515,
            'XCAT':fc.XCAT2,
            'XCAT2':fc.XCAT,
            'Catphan_MTF':fc.Catphan_MTF}

In [4]:
s0 = fc.calculate_spectrum(120, 12, 3, 50)

xx, yy = s0.get_points()

Calculation:  1 / 50
Calculation:  2 / 50
Calculation:  3 / 50
Calculation:  4 / 50
Calculation:  5 / 50
Calculation:  6 / 50
Calculation:  7 / 50
Calculation:  8 / 50
Calculation:  9 / 50
Calculation:  10 / 50
Calculation:  11 / 50
Calculation:  12 / 50
Calculation:  13 / 50
Calculation:  14 / 50
Calculation:  15 / 50
Calculation:  16 / 50
Calculation:  17 / 50
Calculation:  18 / 50
Calculation:  19 / 50
Calculation:  20 / 50
Calculation:  21 / 50
Calculation:  22 / 50
Calculation:  23 / 50
Calculation:  24 / 50
Calculation:  25 / 50
Calculation:  26 / 50
Calculation:  27 / 50
Calculation:  28 / 50
Calculation:  29 / 50
Calculation:  30 / 50
Calculation:  31 / 50
Calculation:  32 / 50
Calculation:  33 / 50
Calculation:  34 / 50
Calculation:  35 / 50
Calculation:  36 / 50
Calculation:  37 / 50
Calculation:  38 / 50
Calculation:  39 / 50
Calculation:  40 / 50
Calculation:  41 / 50
Calculation:  42 / 50
Calculation:  43 / 50
Calculation:  44 / 50
Calculation:  45 / 50
Calculation:  46 / 

In [7]:
s = fc.Spectrum()

contrasts = []
mtfs = []
label = []

fig, axs = plt.subplots(3, 1)

nviews = 480
file_name = 'xcat_phan_mandible_480'
save = False
co = False

for phan in phantoms[:1]:
    phantom = fc.XCAT_mandible()
    for jj in [1,2]:
        for kk in [1,9,-1]:
            spectrum = sorted(spectra)[jj]
            det = sorted(MV_detectors)[kk]
            print(spectrum, det)
            if jj == 1:
                s.x = xx
                s.y = yy
                print('This is kV')
            else:
                s.load(spectrum.split('/')[-1].split('.')[0])
                print('This is MV')

            
            s.attenuate(0.4,fc.get_mu(z=13)) #was 0.3
            s.attenuate(0.089,fc.get_mu(z=22))
            kernel = fc.Detector(s, det.split('/')[-1].split('.')[0])
            kernel.add_focal_spot(1.0)
            angles = np.linspace(0,np.pi*2,nviews,endpoint=False)#480)        
            if not save:
                if jj == 1:
                    phantom.return_projs(kernel,s,angles,det_on=True,convolve_on= co,mgy = 7/nviews,ASG=False,bowtie=True,filter='bowtie_real',load_proj=True,proj_file=file_name)
                else:
                    phantom.return_projs(kernel,s,angles,det_on=True,convolve_on= co,mgy = 7/nviews,ASG=False,bowtie=True,filter='bowtie_real',load_proj=True,proj_file=file_name)
            else:
                if jj == 1:
                    phantom.return_projs(kernel,s,angles,det_on=True,convolve_on= co,mgy = 7/nviews,ASG=False,bowtie=True,filter='bowtie_real',save_proj=True,proj_file=file_name)
                else:
                    phantom.return_projs(kernel,s,angles,det_on=True,convolve_on= co,mgy = 7/nviews,ASG=False,bowtie=True,filter='bowtie_real',save_proj=True,proj_file=file_name)
            
#                 phantom.return_projs(kernel,s,angles,det_on=True,mgy = 7/nviews,convolve_on=False,save_proj=True,proj_file=file_name)
            
            phantom.reconstruct('FDK')
#             phantom.reconstruct('CGLS')
            contrasts.append(phantom.img.T)
            label.append([phan,spectrum,det])
            
#             break
#         break
#     break

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

../../fastcat/data/MV_spectra/Al_spectrum_6.txt ../../fastcat/data/Detectors/CWO-392-micrometer
This is kV
[2021-07-30 16:04:18,923] {simulate.py:214} INFO - Initializing filter bowtie_real
[2021-07-30 16:04:18,923] {simulate.py:219} INFO - Loading attenuations from xcat_phan_mandible_480
[2021-07-30 16:04:18,948] {simulate.py:383} INFO -    Scatter is filtered by bowtie
[2021-07-30 16:04:18,962] {simulate.py:463} INFO -     Interp scatter 512 to 512 pixels
[2021-07-30 16:04:18,962] {simulate.py:463} INFO -     Interp scatter 512 to 512 pixels
[2021-07-30 16:04:19,392] {simulate.py:534} INFO - Loading attenuations is on
[2021-07-30 16:04:19,393] {simulate.py:536} INFO - Running Simulations
[2021-07-30 16:04:19,393] {simulate.py:540} INFO -     1.0 mm focal spot added
[2021-07-30 16:04:19,394] {simulate.py:565} INFO -     Simulating 10 keV
[2021-07-30 16:04:24,257] {simulate.py:565} INFO -     Simulating 20 keV
[2021-07-30 16:04:28,987] {simulate.py:565} INFO -     Simulating 30 keV
[20

In [8]:
np.save('revised_weights_version',contrasts)

In [148]:
plt.figure()
sc = phantom.img

mm = np.nanmean(sc[401:446,307:350,7])
mm_air = np.nanmean(sc[400:,400:,7])

im_HU = (sc[:,:,15] - mm)/(mm_air - mm)*-1000

#     plt.imshow(sc[:,:,15],cmap='gray',vmax=mm+3*ss,vmin=mm-3*ss)
plt.imshow(np.rot90(im_HU,3)[150:390,:],cmap='bone',vmax=240,vmin=-160)
plt.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.5, 9.5, 239.5, -0.5)

In [14]:
np.save('head_scans_no_noise',contrasts)

In [28]:
contrasts = np.load('/home/xcite/fastcat_extra_files/data_second_paper/revised_weights_version.npy')

In [12]:
plt.figure(figsize=[8,4],dpi=300)
import matplotlib.patches as mpatches
# scans = glob('XCATs/scan*')
index = 7

small_titles = ['120 kVp CWO','120 kVp CsI','120 kVp GOS','2.5 MV CWO','2.5 MV CsI','2.5 MV GOS']
alph = ['(a)','(b)','(c)','(d)','(e)','(f)']

h,w = np.rot90(contrasts[0][:,:,index],3)[270:-310,320:-320].shape

X, Y = np.ogrid[:h, :w]
x, y = h//2,w//2

center = [x,y]

dist = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

values = [0.00708,0.008,0.01]
labs = ['Body','Muscle','Skull']

for ii, sc in enumerate(contrasts):
    
    plt.subplot(2,3,ii+1)
    plt.title(small_titles[ii])#scan.split('/')[-1].split('.')[0].split('scan_')[1])

    if ii == 10:
        im = np.fliplr(np.load('../../XCAT_phantom/head_gt.npy'))
        plt.imshow(im,cmap='bone')
        plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)
        
        ax = plt.gca()
        axins = ax.inset_axes([0.65, -0.25, 0.7, 0.9])

        im2 = axins.imshow(im,cmap='bone')

        x1, x2, y1, y2 = 60, 140, 110, 80
        axins.set_xlim(x1, x2)
        axins.set_ylim(y1, y2)
        axins.set_xticklabels('')

        axins.set_xticks([])
        axins.set_yticks([])

        ax.indicate_inset_zoom(axins,ec='red')
        plt.axis('off')
        
        # get the colors of the values, according to the 
        # colormap used by imshow
        colors = [ im2.cmap(im2.norm(value)) for value in values]
        # create a patch (proxy artist) for every color 
        patches = [ mpatches.Patch(color=colors[i], label="{l}".format(l=labs[i]) ) for i in range(len(values)) ]
        # put those patched as legend-handles into the legend
        plt.legend(handles=patches, bbox_to_anchor=(0.65, 1), loc=2, borderaxespad=0. )
    else:
    
        mm = np.mean(sc[385:410,610:640,index])
        mm_air = np.mean(sc[-200:,-200:,index])

        im_HU = (sc[:,:,index] - mm)/(mm_air - mm)*-1000

        im = np.rot90(im_HU,3)[270:-310,320:-320]

        im[dist > 245] = -1000

    #     plt.imshow(sc[:,:,15],cmap='gray',vmax=mm+3*ss,vmin=mm-3*ss)
        plt.imshow(im,cmap='bone',vmax=400,vmin=-400)
        plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)
        plt.axis('off')

        ax = plt.gca()

        axins = ax.inset_axes([0.65, -0.25, 0.7, 0.9])

        axins.imshow(im,cmap='bone',vmax=800,vmin=-0)
        # ax2.set_xlim([-0.05,0.2])
        # ax2.set_ylim([0.05,0.11])
        # sub region of the original image
        x1, x2, y1, y2 = 60, 140, 110, 80
        axins.set_xlim(x1, x2)
        axins.set_ylim(y1, y2)
        axins.set_xticklabels('')

        axins.set_xticks([])
        axins.set_yticks([])

        ax.indicate_inset_zoom(axins,ec='red')

plt.tight_layout()
# plt.savefig('/home/xcite/fastcat_paper_2_tmi/figures/XCATs.pdf') #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Mandible

In [34]:
plt.figure(figsize=[8,4],dpi=300)
import matplotlib.patches as mpatches
# scans = glob('XCATs/scan*')
index = 3

small_titles = ['120 kVp CWO','120 kVp CsI','120 kVp GOS','2.5 MV CWO','2.5 MV CsI','2.5 MV GOS']
alph = ['(a)','(b)','(c)','(d)','(e)','(f)']

h,w = np.rot90(contrasts[0][:,:,index],3)[270:-310,320:-320].shape

X, Y = np.ogrid[:h, :w]
x, y = h//2,w//2

center = [x,y]

dist = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

values = [0.00708,0.008,0.01]
labs = ['Body','Muscle','Skull']

for ii, sc in enumerate(contrasts):
    
    plt.subplot(2,3,ii+1)
    plt.title(small_titles[ii])#scan.split('/')[-1].split('.')[0].split('scan_')[1])

    if ii == 10:
        im = np.fliplr(np.load('../../XCAT_phantom/head_gt.npy'))
        plt.imshow(im,cmap='bone')
        plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)
        
        ax = plt.gca()
        axins = ax.inset_axes([0.65, -0.25, 0.7, 0.9])

        im2 = axins.imshow(im,cmap='bone')

        x1, x2, y1, y2 = 60, 140, 110, 80
        axins.set_xlim(x1, x2)
        axins.set_ylim(y1, y2)
        axins.set_xticklabels('')

        axins.set_xticks([])
        axins.set_yticks([])

        ax.indicate_inset_zoom(axins,ec='red')
        plt.axis('off')
        
        # get the colors of the values, according to the 
        # colormap used by imshow
        colors = [ im2.cmap(im2.norm(value)) for value in values]
        # create a patch (proxy artist) for every color 
        patches = [ mpatches.Patch(color=colors[i], label="{l}".format(l=labs[i]) ) for i in range(len(values)) ]
        # put those patched as legend-handles into the legend
        plt.legend(handles=patches, bbox_to_anchor=(0.65, 1), loc=2, borderaxespad=0. )
    else:
    
        mm = np.mean(sc[485:510,570:600,index])
        mm_air = np.mean(sc[-200:,-200:,index])

        im_HU = (sc[:,:,index] - mm)/(mm_air - mm)*-1000

        im = np.rot90(im_HU,3)[270:-310,320:-320]

        im[dist > 245] = -1000

    #     plt.imshow(sc[:,:,15],cmap='gray',vmax=mm+3*ss,vmin=mm-3*ss)
        plt.imshow(im,cmap='bone',vmax=400,vmin=-400)
        plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)
        plt.axis('off')

        ax = plt.gca()

        axins = ax.inset_axes([0.65, -0.25, 0.7, 0.9])

        axins.imshow(im,cmap='bone',vmax=800,vmin=-0)
        # ax2.set_xlim([-0.05,0.2])
        # ax2.set_ylim([0.05,0.11])
        # sub region of the original image
        x1, x2, y1, y2 = 100, 180, 140, 95
        axins.set_xlim(x1, x2)
        axins.set_ylim(y1, y2)
        axins.set_xticklabels('')

        axins.set_xticks([])
        axins.set_yticks([])

        ax.indicate_inset_zoom(axins,ec='red')

plt.tight_layout()
plt.savefig('/home/xcite/fastcat_paper_2_tmi/figures/XCATs_mandible.pdf') #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2021-07-30 16:22:22,904] {backend_pdf.py:763} DEBUG - Assigning font /b'F1' = '/home/xcite/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'
[2021-07-30 16:22:23,059] {backend_pdf.py:805} DEBUG - Embedding font /home/xcite/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
[2021-07-30 16:22:23,072] {backend_pdf.py:812} DEBUG - Writing TrueType font.


In [16]:
plt.figure()
plt.imshow(sc[:,:,3])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
plt.figure()#dpi=300)

im = np.fliplr(np.load('../../../XCAT_phantom/head_gt.npy'))

im[ im == 0 ] = 0.01
plt.imshow(im,cmap='bone',vmin=0.005,vmax = 0.0083)
# plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)

ax = plt.gca()
axins = ax.inset_axes([0.65, -0.25, 0.7, 0.9])

im2 = axins.imshow(im,cmap='bone',vmin=0.005,vmax = 0.0083)
# ax2.set_xlim([-0.05,0.2])
# ax2.set_ylim([0.05,0.11])
# sub region of the original image
x1, x2, y1, y2 = 60, 140, 110, 80
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')

axins.set_xticks([])
axins.set_yticks([])

ax.indicate_inset_zoom(axins,ec='red')
plt.axis('off')

values = [0.00229,0.00708,0.00769,0.008,0.008,0.0081,0.01]
labs = ['Air','Body','Lymph','Brain','Muscle','Eye Lens','Skull']
# get the colors of the values, according to the 
# colormap used by imshow
colors = [ im2.cmap(im2.norm(value)) for value in values]
# create a patch (proxy artist) for every color 
patches = [ mpatches.Patch(color=colors[i], label="{l}".format(l=labs[i]) ) for i in range(len(values)) ]
# put those patched as legend-handles into the legend
plt.legend(handles=patches, bbox_to_anchor=(0.95, 1), loc=2, borderaxespad=0. )
plt.savefig('/home/xcite/fastcat_paper_2_tmi/figures/XCAT_1.pdf') #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [20]:
plt.figure(figsize=[8,4],dpi=300)
import matplotlib.patches as mpatches
# scans = glob('XCATs/scan*')
index = 7

small_titles = ['120 kVp CsI','2.5 MV CsI']
alph = ['(a)','(b)','(c)','(d)','(e)','(f)']

h,w = np.rot90(contrasts[0][:,:,index],3)[270:-310,320:-320].shape

X, Y = np.ogrid[:h, :w]
x, y = h//2,w//2

center = [x,y]

dist = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

values = [0.00708,0.008,0.01]
labs = ['Body','Muscle','Skull']

for ii, sc in enumerate(contrasts[[1,4]]):
    
    plt.subplot(1,2,ii+1)
    plt.title(small_titles[ii])#scan.split('/')[-1].split('.')[0].split('scan_')[1])

    if ii == 10:
        im = np.fliplr(np.load('../../XCAT_phantom/head_gt.npy'))
        plt.imshow(im,cmap='bone')
        plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)
        
        ax = plt.gca()
        axins = ax.inset_axes([0.65, -0.25, 0.7, 0.9])

        im2 = axins.imshow(im,cmap='bone')
        # ax2.set_xlim([-0.05,0.2])
        # ax2.set_ylim([0.05,0.11])
        # sub region of the original image
        x1, x2, y1, y2 = 60, 140, 110, 80
        axins.set_xlim(x1, x2)
        axins.set_ylim(y1, y2)
        axins.set_xticklabels('')

        axins.set_xticks([])
        axins.set_yticks([])

        ax.indicate_inset_zoom(axins,ec='red')
        plt.axis('off')
        
        # get the colors of the values, according to the 
        # colormap used by imshow
        colors = [ im2.cmap(im2.norm(value)) for value in values]
        # create a patch (proxy artist) for every color 
        patches = [ mpatches.Patch(color=colors[i], label="{l}".format(l=labs[i]) ) for i in range(len(values)) ]
        # put those patched as legend-handles into the legend
        plt.legend(handles=patches, bbox_to_anchor=(0.65, 1), loc=2, borderaxespad=0. )
    else:
    
        mm = np.mean(sc[385:410,610:640,index])
        mm_air = np.mean(sc[-200:,-200:,index])

        im_HU = (sc[:,:,index] - mm)/(mm_air - mm)*-1000

        im = np.rot90(im_HU,3)[270:-310,320:-320]

        im[dist > 245] = -1000

    #     plt.imshow(sc[:,:,15],cmap='gray',vmax=mm+3*ss,vmin=mm-3*ss)
        plt.imshow(im,cmap='bone',vmax=400,vmin=-400)
        plt.text(-70,-20,alph[ii],color='w',fontsize=14)
        plt.axis('off')

#         ax = plt.gca()

#         axins = ax.inset_axes([0.65, -0.25, 0.7, 0.9])

#         axins.imshow(im,cmap='bone',vmax=1000,vmin=-100)
#         # ax2.set_xlim([-0.05,0.2])
#         # ax2.set_ylim([0.05,0.11])
#         # sub region of the original image
#         x1, x2, y1, y2 = 60, 140, 110, 80
#         axins.set_xlim(x1, x2)
#         axins.set_ylim(y1, y2)
#         axins.set_xticklabels('')

#         axins.set_xticks([])
#         axins.set_yticks([])

#         ax.indicate_inset_zoom(axins,ec='red')

plt.tight_layout()
# plt.savefig('/home/xcite/fastcat_paper/figures/XCATs.pdf') #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: list indices must be integers or slices, not list

In [138]:
plt.imshow(sc[401:446,307:350,15],cmap='gray')

In [32]:
from array2gif import write_gif

images = []
for ii in range(phantom.img.T.shape[2]):
    images.append(np.tile(phantom.img.T[:,:,ii]/np.max(phantom.img.T[:,:,ii])*254,[3,1,1]))

write_gif(images, 'rgbbgr.gif', fps=5)

ValueError: The dataset has a value outside the range [0,255]
At position 0 in the list of arrays.

In [33]:
image

NameError: name 'image' is not defined

In [ ]:
images = []
for ii in range(phantom.img.T.shape[2]):
    images.append(imageio.imread(filename))
imageio.mimsave('/path/to/movie.gif', images)

In [19]:
np.save('contrasts',contrasts)
np.save('mtfs',mtfs)
np.save('labels',label)

/home/xcite/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [1]:
%pylab widget
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import fastcat as fc
from glob import glob

## Paper Figure Multipanel with insets

### I'll load all of the kernels 

In [4]:
spectra = glob('../../fastcat/data/MV_spectra/*.txt')
phantoms = ['XCAT','XCAT2','Catphan_404','Catphan_MTF']
MV_detectors = glob('../../fastcat/data/Detectors/*')

dispatcher={'Catphan_404':fc.Catphan_404,
            'XCAT':fc.XCAT2,
            'XCAT2':fc.XCAT,
            'Catphan_MTF':fc.Catphan_MTF}

In [19]:
sorted(spectra)

['../../fastcat/data/MV_spectra/Al_spectrum_25.txt',
 '../../fastcat/data/MV_spectra/Al_spectrum_6.txt',
 '../../fastcat/data/MV_spectra/C_spectrum_25.txt',
 '../../fastcat/data/MV_spectra/C_spectrum_6.txt',
 '../../fastcat/data/MV_spectra/Varian_truebeam.txt',
 '../../fastcat/data/MV_spectra/Varian_truebeam_phasespace.txt',
 '../../fastcat/data/MV_spectra/W_spectrum_25.txt',
 '../../fastcat/data/MV_spectra/W_spectrum_6.txt']

In [76]:
s = fc.Spectrum()



# fig, axs = plt.subplots(3, 1)

nviews = 480#20 #45 
phan_map = [
    'air',
    'water',
    'Spongiosa_Bone_ICRP',
    'G4_LUNG_MD_ICRP',
    'G4_BONE_COMPACT_ICRU',
    'G4_ADIPOSE_TISSUE_ICRP'
    ]

s0 = fc.calculate_spectrum(120, 12, 3, 50)

xx, yy = s0.get_points()

Calculation:  1 / 50
Calculation:  2 / 50
Calculation:  3 / 50
Calculation:  4 / 50
Calculation:  5 / 50
Calculation:  6 / 50
Calculation:  7 / 50
Calculation:  8 / 50
Calculation:  9 / 50
Calculation:  10 / 50
Calculation:  11 / 50
Calculation:  12 / 50
Calculation:  13 / 50
Calculation:  14 / 50
Calculation:  15 / 50
Calculation:  16 / 50
Calculation:  17 / 50
Calculation:  18 / 50
Calculation:  19 / 50
Calculation:  20 / 50
Calculation:  21 / 50
Calculation:  22 / 50
Calculation:  23 / 50
Calculation:  24 / 50
Calculation:  25 / 50
Calculation:  26 / 50
Calculation:  27 / 50
Calculation:  28 / 50
Calculation:  29 / 50
Calculation:  30 / 50
Calculation:  31 / 50
Calculation:  32 / 50
Calculation:  33 / 50
Calculation:  34 / 50
Calculation:  35 / 50
Calculation:  36 / 50
Calculation:  37 / 50
Calculation:  38 / 50
Calculation:  39 / 50
Calculation:  40 / 50
Calculation:  41 / 50
Calculation:  42 / 50
Calculation:  43 / 50
Calculation:  44 / 50
Calculation:  45 / 50
Calculation:  46 / 

In [136]:
contrasts = []
mtfs = []
label = []
ims = []
phan_map = [
    'air',
    'water',
    'Spongiosa_Bone_ICRP',
    '26',
    '26',
    'G4_ADIPOSE_TISSUE_ICRP'
    ]
for phan in phantoms[2:3]:
    phantom = dispatcher[phan](hi_res=False)
    phantom.phan_map = phan_map
#     phantom.geomet.sVoxel = np.array([160,160,160])
#     phantom.geomet.dVoxel = phantom.geomet.sVoxel/phantom.geomet.nVoxel
    for jj in [1,2]:
        for kk in [1,9,-1]:
            spectrum = sorted(spectra)[jj]
            det = sorted(MV_detectors)[kk]
            if jj != 1:
                s.load(spectrum.split('/')[-1].split('.')[0])
                print('This is MV')
                print(spectrum, det)
            else:
                s.x = xx
                s.y = yy
                print('This is kV')
            
            s.attenuate(0.4,fc.get_mu(z=13)) #was 0.3
            s.attenuate(0.089,fc.get_mu(z=22))
#             s.attenuate(0.2,fc.get_mu(z=82))
            kernel = fc.Detector(s, det.split('/')[-1].split('.')[0])
#             kernel.get_plot_mtf_real(axs[2])
#             mtfs.append([kernel.freq,kernel.mtf])
            angles = np.linspace(0,np.pi*2,nviews,endpoint=False)#480)        
            
            if jj == 1:
#                 phantom.return_projs(kernel,s,angles,det_on=True,convolve_on=False,mgy = 7/nviews,ASG=True,bowtie=True,filter='bowtie_real',load_proj=True,proj_file='404_phan_480_160mm')
                phantom.return_projs(kernel,s,angles,det_on=True,convolve_on=False,mgy = 7/nviews,ASG=True,bowtie=True,filter='bowtie_real',load_proj=True,proj_file='404_phan_480_160mm_fe')
            else:
#                 phantom.return_projs(kernel,s,angles,det_on=True,convolve_on=False,mgy = 7/nviews,ASG=False,bowtie=True,filter='bowtie_real',load_proj=True,proj_file='404_phan_480_160mm')
                phantom.return_projs(kernel,s,angles,det_on=True,convolve_on=False,mgy = 7/nviews,ASG=False,bowtie=True,filter='bowtie_real',load_proj=True,proj_file='404_phan_480_160mm_fe')
                
#             phantom.return_projs(kernel,s,angles,det_on=True,convolve_on=False,mgy = 7/nviews,ASG=True,bowtie=True,filter='bowtie_real',load_proj=True,proj_file='404_phan_480_160mm_fe')
#             else:
#                 phantom.return_projs(kernel,s,angles,det_on=True,mgy = 7/nviews,convolve_on=False,load_proj=True,proj_file='404_phan_480_160mm')
            
            phantom.reconstruct('FDK')
#             phantom.reconstruct('CGLS')
            contrasts.append(phantom.img.T)
            label.append([phan,spectrum,det])
            
#             break
#         break
#     break

[2021-07-29 10:09:29,718] {phantoms.py:650} INFO - Phantom is low resolution
This is kV
[2021-07-29 10:09:29,753] {simulate.py:214} INFO - Initializing filter bowtie_real
[2021-07-29 10:09:29,754] {simulate.py:219} INFO - Loading attenuations from 404_phan_480_160mm_fe
[2021-07-29 10:09:29,770] {simulate.py:415} INFO - Initializing ASG
[2021-07-29 10:09:29,787] {simulate.py:534} INFO - Loading attenuations is on
[2021-07-29 10:09:29,788] {simulate.py:536} INFO - Running Simulations
[2021-07-29 10:09:29,788] {simulate.py:565} INFO -     Simulating 10 keV
[2021-07-29 10:09:30,467] {simulate.py:565} INFO -     Simulating 20 keV
[2021-07-29 10:09:31,148] {simulate.py:565} INFO -     Simulating 30 keV
[2021-07-29 10:09:31,832] {simulate.py:565} INFO -     Simulating 40 keV
[2021-07-29 10:09:32,767] {simulate.py:565} INFO -     Simulating 50 keV
[2021-07-29 10:09:34,092] {simulate.py:565} INFO -     Simulating 60 keV
[2021-07-29 10:09:35,473] {simulate.py:565} INFO -     Simulating 70 keV
[2

In [135]:
plt.figure(figsize=[8,4],dpi=300)
import matplotlib.patches as mpatches
# scans = glob('XCATs/scan*')
index = 4

small_titles = ['120 kVp CWO','120 kVp CsI','120 kVp GOS','2.5 MV CWO','2.5 MV CsI','2.5 MV GOS']
alph = ['(a)','(b)','(c)','(d)','(e)','(f)']

h,w = np.rot90(contrasts[0][:,:,index],3)[270:-310,320:-320].shape

X, Y = np.ogrid[:h, :w]
x, y = h//2,w//2

center = [x,y]

dist = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

values = [0.00708,0.008,0.01]
labs = ['Body','Muscle','Skull']

for ii, sc in enumerate(contrasts):
        
    plt.subplot(2,3,ii+1)
    plt.title(small_titles[ii])#scan.split('/')[-1].split('.')[0].split('scan_')[1])

    mm = np.mean(sc[376:406,376:406,index])
    mm_air = np.mean(sc[-50:,-50:,index])

    im_HU = (sc[:,:,4] - mm)/(mm_air - mm)*-1000

    im = np.rot90(im_HU,3)#[270:-310,320:-320]

#         im[dist > 245] = -1000

#     plt.imshow(sc[:,:,15],cmap='gray',vmax=mm+3*ss,vmin=mm-3*ss)
    plt.imshow(im,cmap='bone',vmax=400,vmin=-400)
    plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)
    plt.axis('off')

    ax = plt.gca()

    axins = ax.inset_axes([0.65, -0.10, 0.5, 0.5])

    axins.imshow(im,cmap='bone',vmax=400,vmin=-400)
    # ax2.set_xlim([-0.05,0.2])
    # ax2.set_ylim([0.05,0.11])
    # sub region of the original image
    x1, x2, y1, y2 = 215, 295, 480, 410
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_xticklabels('')

    axins.set_xticks([])
    axins.set_yticks([])

    lines = ax.indicate_inset_zoom(axins,ec='red')
    lines[1][0].set_visible(False)
    lines[1][1].set_visible(False)
    lines[1][2].set_visible(True)
    lines[1][3].set_visible(True)
    

plt.tight_layout()
# plt.savefig('/home/xcite/fastcat_paper_2_tmi/figures/contrasts.pdf') #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2021-07-29 10:06:22,103] {backend_pdf.py:763} DEBUG - Assigning font /b'F1' = '/home/xcite/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'
[2021-07-29 10:06:22,213] {backend_pdf.py:805} DEBUG - Embedding font /home/xcite/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
[2021-07-29 10:06:22,214] {backend_pdf.py:812} DEBUG - Writing TrueType font.


In [102]:
np.save('contrasts_fe_artifacts',contrasts)

# Metal artifact image

In [138]:
plt.figure(figsize=[8,4],dpi=300)
import matplotlib.patches as mpatches
# scans = glob('XCATs/scan*')
index = 4

small_titles = ['120 kVp CWO','120 kVp CsI','120 kVp GOS','2.5 MV CWO','2.5 MV CsI','2.5 MV GOS']
alph = ['(a)','(b)','(c)','(d)','(e)','(f)']

h,w = np.rot90(contrasts[0][:,:,index],3)[270:-310,320:-320].shape

X, Y = np.ogrid[:h, :w]
x, y = h//2,w//2

center = [x,y]

dist = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

values = [0.00708,0.008,0.01]
labs = ['Body','Muscle','Skull']

for ii, sc in enumerate(contrasts):
        
    plt.subplot(2,3,ii+1)
    plt.title(small_titles[ii])#scan.split('/')[-1].split('.')[0].split('scan_')[1])

    mm = np.mean(sc[376:406,376:406,index])
    mm_air = np.mean(sc[-50:,-50:,index])

    im_HU = (sc[:,:,4] - mm)/(mm_air - mm)*-1000

    im = np.rot90(im_HU,3)#[270:-310,320:-320]

#         im[dist > 245] = -1000

#     plt.imshow(sc[:,:,15],cmap='gray',vmax=mm+3*ss,vmin=mm-3*ss)
    plt.imshow(im,cmap='bone',vmax=400,vmin=-400)
    plt.text(-70,-20,alph[ii],color='cornflowerblue',fontsize=14)
    plt.axis('off')

    ax = plt.gca()

    axins = ax.inset_axes([0.65, -0.10, 0.5, 0.5])

    axins.imshow(im,cmap='bone',vmax=1500,vmin=-500)
    # ax2.set_xlim([-0.05,0.2])
    # ax2.set_ylim([0.05,0.11])
    # sub region of the original image
    x1, x2, y1, y2 =  410, 480 ,295, 215
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_xticklabels('')

    axins.set_xticks([])
    axins.set_yticks([])

    ax.indicate_inset_zoom(axins,ec='red')

plt.tight_layout()
# plt.savefig('/home/xcite/fastcat_paper_2_tmi/figures/contrasts_bone.pdf') #

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …